In [1]:
import os
from typing import Iterable, Tuple, Sequence
import numpy as np

def generate_folder_path(dirs: Iterable[str]) -> None:
    """
    Create each directory in `dirs` if it does not already exist.

    Parameters
    ----------
    dirs : Iterable[str]
        Paths to create.

    Notes
    -----
    - Uses `os.makedirs(..., exist_ok=True)` so it's safe to call repeatedly.
    - Prints a short status line per directory (matching your original behavior).
    """
    for d in dirs:
        try:
            os.makedirs(d, exist_ok=True)
            print("Directory", d, "created or already exists")
        except OSError as e:
            # Catch broader OS errors (permissions, invalid paths, etc.)
            print("Failed to create directory", d, "—", str(e))


def load_s2p_files(dp_s2p: str, neuropil_correction: float):
    """
    Load Suite2p output files, with optional neuropil correction.

    Parameters
    ----------
    dp_s2p : str
        Path to the Suite2p output directory containing:
        F.npy, Spks.npy, ops.npy, iscell.npy, stat.npy, Fneu.npy
    neuropil_correction : float
        If > 0, apply: F_corrected = F - neuropil_correction * Fneu,
        then baseline-shift each ROI to start at 0 (subtract per-ROI min).

    Returns
    -------
    F : np.ndarray
        (n_rois, n_timepoints) fluorescence (corrected if requested).
    Spks : np.ndarray
        (n_rois, n_timepoints) deconvolved spikes (Suite2p output).
    ops : dict
        Suite2p ops dictionary.
    iscell : np.ndarray
        (n_rois, 2) first col is 0/1 (not cell / cell), second is probability.
    stat : np.ndarray
        Array of ROI dicts with Suite2p ROI stats (allow_pickle=True).
    Fneu : np.ndarray
        (n_rois, n_timepoints) neuropil traces.

    Notes
    -----
    - Avoids `os.chdir`; loads with absolute paths.
    """
    F = np.load(os.path.join(dp_s2p, "F.npy"))
    Spks = np.load(os.path.join(dp_s2p, "Spks.npy"))
    ops = np.load(os.path.join(dp_s2p, "ops.npy"), allow_pickle=True).item()
    iscell = np.load(os.path.join(dp_s2p, "iscell.npy"))
    stat = np.load(os.path.join(dp_s2p, "stat.npy"), allow_pickle=True)
    Fneu = np.load(os.path.join(dp_s2p, "Fneu.npy"))

    if neuropil_correction > 0:
        F = F - Fneu * neuropil_correction
        # Shift each ROI to be non-negative (subtract per-ROI min)
        F = F - F.min(axis=1, keepdims=True)

    return F, Spks, ops, iscell, stat, Fneu


def pre_process_imaging(
    iscell: np.ndarray,
    F: np.ndarray,
    stat: np.ndarray,
    FOVsizeum: float,
    mode: str
):
    """
    Compute dF/F for accepted cells and extract their centroids.

    Parameters
    ----------
    iscell : np.ndarray
        Suite2p iscell array (n_rois, 2). Column 0 is 0/1 mask.
    F : np.ndarray
        Fluorescence traces (n_rois, n_timepoints).
    stat : np.ndarray
        ROI stats from Suite2p (array of dicts with 'med' field).
    FOVsizeum : float
        Size of the field of view in micrometers (assumes Suite2p 512 px grid).
    mode : {'median', '10'}
        Baseline strategy for dF/F:
        - 'median': baseline = median(trace)
        - '10'    : baseline = median of lowest 10% of samples

    Returns
    -------
    FNc : np.ndarray
        dF/F traces for curated cells (n_cells, n_timepoints).
    iscell_list : np.ndarray
        Indices of curated cells.
    xa, yb : list[float]
        Centroid coordinates in micrometers.
    x, y : tuple[list[float], list[float]]
        Centroid coordinates in pixels (x, y).

    Notes
    -----
    - Uses a pixel->µm scale of FOVsizeum/512 (matches your original).
    """
    iscell_list = get_curated_cells(iscell)
    Fc = F[iscell_list]

    if mode == 'median':
        FNc = dff_median(Fc)
    elif mode == '10':
        FNc = dff_10percent(Fc)
    else:
        raise ValueError("mode must be 'median' or '10'")

    x, y = get_cell_centroids(stat, iscell_list)
    scale = FOVsizeum / 512.0
    xa = [xi * scale for xi in x]
    yb = [yi * scale for yi in y]

    return FNc, iscell_list, xa, yb, x, y


def get_cell_centroids(stat: np.ndarray, index_list: Sequence[int]) -> Tuple[list, list]:
    """
    Extract (x, y) centroids (in pixels) for the given ROI indices from Suite2p `stat`.

    Parameters
    ----------
    stat : np.ndarray
        Array of ROI dicts; each must contain 'med' as (y, x).
    index_list : Sequence[int]
        ROI indices to extract.

    Returns
    -------
    x, y : list, list
        Lists of x and y coordinates (pixels).
    """
    # Suite2p stores 'med' as (y, x); you returned x=med[1], y=med[0]
    x, y = zip(*[(stat[i]['med'][1], stat[i]['med'][0]) for i in index_list])
    return list(x), list(y)


def get_curated_cells(iscell: np.ndarray) -> np.ndarray:
    """
    Return indices of ROIs labeled as cells by Suite2p.

    Parameters
    ----------
    iscell : np.ndarray
        (n_rois, 2) array where column 0 is 0/1 (not cell / cell).

    Returns
    -------
    np.ndarray
        1D array of indices where iscell[:, 0] == 1.
    """
    return np.where(iscell[:, 0] == 1)[0]


def dff_10percent(traces: np.ndarray) -> np.ndarray:
    """
    Compute dF/F using the median of the lowest 10% of samples as baseline.

    Parameters
    ----------
    traces : np.ndarray
        (n_rois, n_timepoints) array.

    Returns
    -------
    np.ndarray
        dF/F with same shape as `traces`.

    Notes
    -----
    - Uses a robust baseline (median of bottom 10%).
    - Adds a tiny epsilon to avoid division by zero.
    """
    traces = np.asarray(traces)
    out = np.empty_like(traces, dtype=float)
    n = traces.shape[1]
    k = max(1, int(n / 10))
    eps = 1e-12

    for idx, row in enumerate(traces):
        # indices of k smallest values (unordered)
        kth_idx = np.argpartition(row, k - 1)[:k]
        bsl = float(np.median(row[kth_idx]))
        out[idx] = (row - bsl) / (bsl + eps)
    return out


def dff_median(traces: np.ndarray) -> np.ndarray:
    """
    Compute dF/F using the median as baseline.

    Parameters
    ----------
    traces : np.ndarray
        (n_rois, n_timepoints) array.

    Returns
    -------
    np.ndarray
        dF/F with same shape as `traces`.
    """
    traces = np.asarray(traces)
    eps = 1e-12
    med = np.median(traces, axis=1, keepdims=True)
    return (traces - med) / (med + eps)
